- Author: Chandresh Kumar
- Date: 2025-07-10

In [1]:
# !pip install -U "vectorbtpro[base] @ git+https://${GITHUB_TOKEN}@github.com/polakowo/vectorbt.pro.git"

In [2]:
# Upgrade Plotly and Kaleido for compatibility
!pip install --upgrade plotly>=6.1.1 kaleido==0.2.1

zsh:1: 6.1.1 not found


In [3]:
from vectorbtpro import *  

data = vbt.BinanceData.pull('BTCUSDT')

/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
print(data)


BinanceData(
    wrapper=ArrayWrapper(
        index=<pandas.core.indexes.datetimes.DatetimeIndex object at 0x157ae1580 with shape (2885,)>,
        columns=<pandas.core.indexes.base.Index object at 0x310695460 with shape (9,)>,
        ndim=2,
        freq='1d',
        parse_index=None,
        column_only_select=None,
        range_only_select=None,
        group_select=None,
        grouped_ndim=None,
        grouper=Grouper(
            index=<pandas.core.indexes.base.Index object at 0x310695460 with shape (9,)>,
            group_by=None,
            def_lvl_name='group',
            allow_enable=True,
            allow_disable=True,
            allow_modify=True
        )
    ),
    data=symbol_dict(
        BTCUSDT=<pandas.core.frame.DataFrame object at 0x1098cb130 with shape (2885, 9)>
    ),
    single_key=True,
    classes=symbol_dict(),
    level_name=None,
    fetch_kwargs=symbol_dict(
        BTCUSDT=dict(
            silence_warnings=False
        )
    ),
    returned_k

In [5]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import yfinance as yf

# Load sample data (Example: Apple stock - AAPL)
symbol = 'AAPL'
data_df = yf.download(symbol, start='2022-01-01', end='2025-01-01')
if data_df is None or 'Close' not in data_df.columns:
    raise ValueError("Failed to download data or 'Close' column not found.")
data = data_df['Close']

# Calculate Moving Averages using pandas rolling
short_window = 20
long_window = 50

ma_short = data.rolling(window=short_window).mean()
ma_long = data.rolling(window=long_window).mean()

# Generate Buy/Sell Signals (detect crossovers)
entries = (ma_short > ma_long) & (ma_short.shift(1) <= ma_long.shift(1))
exits = (ma_short < ma_long) & (ma_short.shift(1) >= ma_long.shift(1))

# Portfolio Simulation
portfolio = vbt.Portfolio.from_signals(
    close=data,
    entries=entries,
    exits=exits,
    init_cash=10000,
    fees=0.001
)

# Print Results
print(portfolio.stats())

# Plot the Portfolio Performance


/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/vectorbt/root_accessors.py:108: UserWarning: registration of accessor <class 'vectorbt.root_accessors.Vbt_SRAccessor'> under name 'vbt' for type <class 'pandas.core.series.Series'> is overriding a preexisting attribute with the same name.
  class Vbt_SRAccessor(DirNamesMixin, GenericSRAccessor):
/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/vectorbt/root_accessors.py:119: UserWarning: registration of accessor <class 'vectorbt.root_accessors.Vbt_DFAccessor'> under name 'vbt' for type <class 'pandas.core.frame.DataFrame'> is overriding a preexisting attribute with the same name.
  class Vbt_DFAccessor(DirNamesMixin, GenericDFAccessor):
/var/folders/tj/z_3mk9tx70z6dm5t951hflnc0000gp/T/ipykernel_99354/3969596799.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_df = yf.download(symbol, start='2022-01-01', end='2025-01-01')
[***************

Start                         2022-01-03 00:00:00
End                           2024-12-31 00:00:00
Period                                        753
Start Value                               10000.0
End Value                            11079.758931
Total Return [%]                        10.797589
Benchmark Return [%]                    39.839619
Max Gross Exposure [%]                      100.0
Total Fees Paid                        157.656632
Max Drawdown [%]                        29.458132
Max Drawdown Duration                       469.0
Total Trades                                  9.0
Total Closed Trades                           8.0
Total Open Trades                             1.0
Open Trade PnL                         333.045173
Win Rate [%]                                 50.0
Best Trade [%]                          24.039584
Worst Trade [%]                        -10.936992
Avg Winning Trade [%]                    9.874204
Avg Losing Trade [%]                    -6.895381


/var/folders/tj/z_3mk9tx70z6dm5t951hflnc0000gp/T/ipykernel_99354/3969596799.py:34: UserWarning: Object has multiple columns. Aggregating using <function mean at 0x109a24d30>. Pass column to select a single column/group.
  print(portfolio.stats())


In [6]:
# Plot the Portfolio Performance using fallback only
import plotly.graph_objects as go

# Get portfolio value over time
portfolio_value = portfolio.value()

fig = go.Figure()
# Add portfolio value trace
fig.add_trace(go.Scatter(
    x=portfolio_value.index,
    y=portfolio_value.values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='blue', width=2)
))
# Add stock price for reference on secondary axis
fig.add_trace(go.Scatter(
    x=data.index,
    y=data.values,
    mode='lines',
    name='Stock Price',
    line=dict(color='gray', width=1),
    yaxis='y2'
))
# Plot buy/sell signals
buy_dates = entries[entries].index
sell_dates = exits[exits].index

if len(buy_dates) > 0:
    buy_prices = data.loc[buy_dates]
    fig.add_trace(go.Scatter(
        x=buy_dates,
        y=buy_prices.values,
        mode='markers',
        name='Buy Signals',
        marker=dict(color='green', size=12, symbol='triangle-up'),
        yaxis='y2'
    ))
if len(sell_dates) > 0:
    sell_prices = data.loc[sell_dates]
    fig.add_trace(go.Scatter(
        x=sell_dates,
        y=sell_prices.values,
        mode='markers',
        name='Sell Signals',
        marker=dict(color='red', size=12, symbol='triangle-down'),
        yaxis='y2'
    ))
fig.update_layout(
    title='Portfolio Performance with Trading Signals',
    xaxis_title='Date',
    yaxis=dict(title='Portfolio Value ($)', side='left'),
    yaxis2=dict(title='Stock Price ($)', side='right', overlaying='y'),
    hovermode='x unified',
    showlegend=True
)
fig.show()

/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/kaleido/__init__.py:14: UserWarning:




This means that static image generation (e.g. `fig.write_image()`) will not work.

Please upgrade Plotly to version 6.1.1 or greater, or downgrade Kaleido to version 0.2.1.




In [8]:
# Use existing data, ma_short, ma_long from previous cells

# Generate Buy/Sell Signals (detect crossovers)
entries = (ma_short > ma_long) & (ma_short.shift(1) <= ma_long.shift(1))
exits = (ma_short < ma_long) & (ma_short.shift(1) >= ma_long.shift(1))

# Select the single column (AAPL) as Series for vectorbt
close = data['AAPL']
entries_series = entries['AAPL']
exits_series = exits['AAPL']

# Portfolio Simulation using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=close,
    entries=entries_series,
    exits=exits_series,
    init_cash=10000,
    fees=0.001
)

# Print Results
print(portfolio.stats())

# Plot the Portfolio Performance
portfolio.plot()


/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/chandreshkumar/Desktop/code/fin-tests/.venv/lib/python3.9/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2022-01-03 00:00:00
End                           2024-12-31 00:00:00
Period                                        753
Start Value                               10000.0
End Value                            11079.758931
Total Return [%]                        10.797589
Benchmark Return [%]                    39.839619
Max Gross Exposure [%]                      100.0
Total Fees Paid                        157.656632
Max Drawdown [%]                        29.458132
Max Drawdown Duration                       469.0
Total Trades                                    9
Total Closed Trades                             8
Total Open Trades                               1
Open Trade PnL                         333.045173
Win Rate [%]                                 50.0
Best Trade [%]                          24.039584
Worst Trade [%]                        -10.936992
Avg Winning Trade [%]                    9.874204
Avg Losing Trade [%]                    -6.895381


FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': '7dff109b-4d6b-411d-ae3a-5c4a44f2e2d8',
              'x': array([datetime.datetime(2022, 1, 3, 0, 0),
                          datetime.datetime(2022, 1, 4, 0, 0),
                          datetime.datetime(2022, 1, 5, 0, 0), ...,
                          datetime.datetime(2024, 12, 27, 0, 0),
                          datetime.datetime(2024, 12, 30, 0, 0),
                          datetime.datetime(2024, 12, 31, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([178.64562988, 176.37837219, 171.68670654, ..., 254.97492981,
                          251.59307861, 249.81736755]),
              'yaxis': 'y'},
             {'customdata': array([[ 0.        , 58.06692149,  9.99000999],
                                   [ 2.        , 60.30090279,  9.08